# FeatureFlex 0.1.0
### Autorzy: Andrii Voznesenskyi, Bartosz Kaczorowski

**FeatureFlex** to pakiet AutoML skierowany do badaczy i specjalistów zajmujących się systemami rekomendacji opartymi na głębokim uczeniu. Odpowiada on na problem efektywnego zarządzania cechami wejściowymi w dużych zestawach danych. Dzięki temu może stanowić świetne narzędzie dla firm e-commerce lub platform streamingowych, które wykorzystają systemy rekomendacji na co dzień.

**FeatureFlex** specjalizuje się w:
- automatycznym wyborze najistotniejszych cech dla danego zbioru,
- zwiększaniu efektywności systemów rekomendacji,
- poprawie dokładności przewidywania preferencji podczas analizy predykcyjnej.

Pakiet inspirowany był pracą [*AutoField: Automating Feature Selection in Deep Recommender Systems*](https://arxiv.org/pdf/2204.09078). Autorzy artykułu oprócz podania własnego rozwiązania rozważają tam między innymi metody takie jak:
- manualna selekcja - ta wymaga jednak nakładów ludzkich i wiedzy eksperckiej,
- Grid/Random Search - porównywalne jeśli chodzi o dokładność, jednak słabo skalujące się i będące wymagające obliczeniowo,
- Lasso/Decision Trees - choć czasami stosowane, nie sprawdzają się jednak dobrze dla głębokich systemów rekomendacji.

Stąd idea stworzenia systemu AutoML, który odpowie na powyższe problemy.




## Zawartość pakietu

Przepływ pracy z danymi można podzielić na trzy cześci, za które odpowiedzialne są następujące klasy:
1. Przetwarzanie danych (preprocessing):
- `DataPreprocessor` - przetwarza dane z zadanego zbioru przy pomocy metody `preprocess`, opcjonalnie pozwala na manualny wybór cech.

2. Selekcja i optymalizacja modeli:
- `EnhancedFeatureSelector` - wybiera cechy przy użyciu trenowalnego modelu, udostępnia dwie metody wyboru najistotniejszych cech:
     - `select_via_shap` - wykorzystuje pakiet ([SHapley Additive exPlanations](https://shap.readthedocs.io/en/latest/)) do wyboru cech, których ważność ustalana jest przy użyciu lasu losowego,
     - `select_via_model_optimizer` - stosuje własny model AutoML do wyboru cech (patrz poniżej).

- `ModelOptimizer` - automatycznie wybiera i optymalizuje modele uczenia maszynowego przy pomocy metody `optimize_model`, stosuje do tego *Grid Search* wykorzystując pole pod krzywą ROC i własny mechanizm kroswalidacji. Modele poddawane przeszukiwaniu w obecnej wersji: *RandomForest*, *GradientBoosting*, *LogisticRegression*, *SVM*, *XGBoost*, *KNN*.

3. Ewaluacja i podsumowanie wyników:
- `ModelEvaluator` - oblicza metryki, rysuje wykresy i generuje raporty w formie konsolowej lub pliku HTML dla wybranego modelu. Wybrane metryki: *AUC*, *Accuracy*, *Precission*, *Recall*, *F1-Score*.

In [3]:
import sys, os

current_dir = os.path.dirname(os.path.abspath("__file__"))
src_path = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(src_path)

## Przykładowy przepływ pracy

### Wybór zbioru i przetwarzanie danych

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from preprocessing import DataPreprocessor

# Wybór zbioru danych
dataset = "../../data/world-happiness-report-2021.csv"
data = pd.read_csv(dataset)
print("Wczytano zbiór: world-happiness-report-2021.csv")

# Określenie zbioru cech
columns = [
    "Country name", "Regional indicator", "Ladder score", "Logged GDP per capita", 
    "Social support", "Healthy life expectancy", "Freedom to make life choices", 
    "Generosity", "Perceptions of corruption"
]
data = data[columns]

target_column = "Ladder score"
data[target_column] = (data[target_column] > data[target_column].mean()).astype(int)

# Przetwarzanie danych
preprocessor = DataPreprocessor()
X, y, _ = preprocessor.preprocess(
    data,               # zbiór danych w postaci ramki
    target_column       # nazwa kolumny zawierającej etykiety
)

# Podział przetworzonego zbioru na treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Wczytano zbiór: world-happiness-report-2021.csv


### Wybór najistotniejszych cech

In [5]:
from feature_selector import EnhancedFeatureSelector

selector = EnhancedFeatureSelector(
    input_dim=X_train.shape[1]  # liczba kolumn
)

# Wybór liczby i metody ekstrakcji cech
n_features = 10
# top_features = selector.select_via_shap(X_train, y_train, n_features=n_features)
top_features = selector.select_via_model_optimizer(X_train, y_train, n_features=n_features)

reduced_selector = EnhancedFeatureSelector(input_dim=len(top_features))

# Ograniczenie zbioru danych do wyekstrahowanych cech
if hasattr(X_train, "toarray"):
    X_train_dense = X_train.toarray()[:, top_features]
    X_test_dense = X_test.toarray()[:, top_features]
else:
    X_train_dense = X_train[:, top_features]
    X_test_dense = X_test[:, top_features]

/home/bakaczor/Desktop/Projects/AutoML/2/CS-MINI-2024Z-AutoML_project_2/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Model Optimization: 100%|██████████| 6/6 [00:05<00:00,  1.06it/s]


### Wybór i optymalizacja modelu

In [6]:
from imblearn.combine import SMOTETomek
from model_optimizer import ModelOptimizer
    
# Opcjonalnie: redukcja nierówności powstałych w zbiorze,
# stosując technikę oversamplingu SMOTE-Tomek
smote_tomek = SMOTETomek(random_state=42)
X_train_res, y_train_res = smote_tomek.fit_resample(X_train_dense, y_train)

# Wybór i optymalizacja modelu
optimizer = ModelOptimizer()
best_model, best_score = optimizer.optimize_model(
    X_train_res,    # macierz cech
    y_train_res     # wektor etykiet
)
print(f"Best Model Score (CV AUC): {best_score}")

Model Optimization: 100%|██████████| 6/6 [00:05<00:00,  1.18it/s]

Best Model Score (CV AUC): 0.9779316712834719


### Ewaluacja uzyskanego modelu

In [7]:
from evaluation import ModelEvaluator

evaluator = ModelEvaluator()
evaluation_results = evaluator.evaluate(
    best_model,                     # wybrany model
    X_test_dense,                   # macierz cech
    y_test,                         # wektor etykiet
    output_format="console",        # metoda prezentacji - "console" lub "html"
)
print("Evaluation Results:", evaluation_results)

Evaluating model predictions...

=== Evaluation Metrics ===
AUC: 0.9420
Accuracy: 0.9000
Precision: 0.9333
Recall: 0.8750
F1-Score: 0.9032

=== Confusion Matrix ===
[[13  1]
 [ 2 14]]

=== Classification Report ===
              precision    recall  f1-score   support

           0     0.8667    0.9286    0.8966        14
           1     0.9333    0.8750    0.9032        16

    accuracy                         0.9000        30
   macro avg     0.9000    0.9018    0.8999        30
weighted avg     0.9022    0.9000    0.9001        30

Evaluation Results: {'AUC': 0.9419642857142857, 'Accuracy': 0.9, 'Precision': 0.9333333333333333, 'Recall': 0.875, 'F1-Score': 0.9032258064516129}


### Automatyzacja
W pliku `main.py` dostępna jest funkcja pozwalająca na automatyzację całego przedstawionego procesu. Użyjemy jej do wygenerowania raportu w postaci HTML.

In [8]:
from main import preprocess_and_train
from IPython.display import HTML

preprocess_and_train(data, target_column=target_column, output_filename="evaluation_report_happiness.html", output_path=".")
HTML(filename="./evaluation_report_happiness.html")

Preprocessing data...
Splitting data into train and test sets...
Performing feature selection...


Model Optimization: 100%|██████████| 6/6 [00:05<00:00,  1.12it/s]


Applying SMOTE-Tomek on the training set...
Optimizing models...


Model Optimization: 100%|██████████| 6/6 [00:04<00:00,  1.31it/s]


Best Model Score (CV AUC): 0.9779316712834719
Evaluating model...
Evaluating model predictions...
HTML report saved to: ./evaluation_report_happiness.html
Evaluation Results: {'AUC': 0.9419642857142857, 'Accuracy': 0.9, 'Precision': 0.9333333333333333, 'Recall': 0.875, 'F1-Score': 0.9032258064516129}
